In [1]:
# test out the database

# think about how to set up the contextual embedding training pipeline



In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import Row, StructType, StructField, IntegerType, StringType

import os
import time
import random
#import mysql.connector as mysql   # import gc




In [3]:
SUBMIT_ARGS = "--driver-class-path file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar --jars file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

db_name = 'test_pubmed'  # db name collisons? https://stackoverflow.com/questions/14011968/user-cant-access-a-database
table_name = 'abstracts' # 'abstracts'

url = "jdbc:mysql://localhost:3306/{}?useUnicode=true&useJDBCCompliantTimezoneShift=true&useLegacyDatetimeCode=false&serverTimezone=America/Chicago".format(db_name)  # mysql runs on port 3306


In [4]:
print('initializing spark')
# init spark
conf = SparkConf()
conf = (conf.setMaster('local[*]')
       .set('spark.driver.memory','24G')
       .set("spark.jars", "/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar"))        
'''
.set('spark.executor.memory','1G')  # 20
.set('spark.driver.memory','1G')   # 40
.set('spark.driver.maxResultSize','500M')  #.set('spark.storage.memoryFraction',0))  # this setting is now a legacy option
.set('spark.python.worker.reuse', 'false')
.set('spark.python.worker.memory','512m')
.set('spark.executor.cores','1'))
'''
sc = SparkContext(conf=conf)
#sc.addJar('home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar')  # temp
spark = SparkSession(sc)  # don't need this for vanilla RDDs

print(sc._conf.getAll())

initializing spark
[('spark.driver.memory', '24G'), ('spark.repl.local.jars', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.app.id', 'local-1563565969652'), ('spark.jars', '/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.driver.port', '41993'), ('spark.executor.id', 'driver'), ('spark.app.name', 'pyspark-shell'), ('spark.rdd.compress', 'True'), ('spark.driver.extraClassPath', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.serializer.objectStreamReset', '100'), ('spark.driver.host', 'midway2-0085.rcc.local'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]


In [27]:
# test read

sql = "(SELECT * FROM {} LIMIT 100000) AS t".format(table_name)
print(sql)

start_time = time.time()
df = spark.read.format('jdbc').option("url", url)\
                              .option("dbtable", sql)\
                              .load().repartition(9).cache()
end_time = time.time()
print("dataframe loaded in {} s".format(end_time - start_time))

print(df.rdd.getNumPartitions())

(SELECT * FROM abstracts LIMIT 100000) AS t
dataframe loaded in 0.06559634208679199 s
9


In [30]:
start_time = time.time()
test_subset = df.take(3)
end_time = time.time()

for entry in test_subset:
    print(entry)
    print()
print()
print("show operation. elapsed: {} s".format(end_time - start_time))



Row(pmid='17616371', title='Preventing the self-adherence of Mefix.', abstract='')

Row(pmid='17532617', title='Closed cycle construction: an integrated process for the separation and reuse of C&D waste.', abstract="In The Netherlands, construction and demolition (C&D) waste is already to a large extent being reused, especially the stony fraction, which is crushed and reused as a road base material. In order to increase the percentage of reuse of the total C&D waste flow to even higher levels, a new concept has been developed. In this concept, called 'Closed Cycle Construction', the processed materials are being reused at a higher quality level and the quantity of waste that has to be disposed of is minimised. For concrete and masonry, the new concept implies that the material cycle will be completely closed, and the original constituents (clay bricks, gravel, sand, cement stone) are recovered in thermal processes. The mixed C&D waste streams are separated and decontaminated. For this 

In [ ]:
sc.stop()